<a href="https://colab.research.google.com/github/bhosalemahesh899/AIPractice/blob/main/openai/gpt/TextClassificationFineTune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Funetune GPT 3.5 Model in open ai

- This link is source for reference
https://github.com/PradipNichite/Youtube-Tutorials/blob/main/GPT3.5%20Finetuning/GPT_3_5_Finetuning_Tutorial.ipynb



In [1]:
!pip install -U openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.3/389.3 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2


In [3]:
import pandas as pd
df = pd.read_json("/content/data_all_combined.json")
df.head()

,Category,SubCategory,Keywords,Answer
0,Assortment / Products,Product availability,"[limited stock, out of stock, availability iss...","I was looking for a specific product, but it w..."
1,Assortment / Products,Search results relevance,"[irrelevant search, hard to find, confusing re...",The search results didn�t seem relevant to my ...
2,Customer Service,Professionalism,"[unprofessional, not helpful, rude representat...",The customer service experience was subpar. Th...
3,Customer Service,Responsiveness,"[slow response, delayed replies, not timely]","I reached out for assistance with my order, bu..."
4,Website / App,Mobile responsiveness,"[mobile issues, not responsive, poor mobile ex...","The website works fine on a desktop, but the m..."


In [4]:
cats = pd.read_json("/content/categories.json")
cats.head()

catsStr = str(cats)

In [29]:
def convert_to_gpt35_format2(sysIns, dataset):
    fine_tuning_data = []
    for _, row in dataset.iterrows():
        result = ", ".join(row['Keywords'])
        #print(result)
        print(str(row['Keywords']))

    return fine_tuning_data

In [ ]:
sysIns = 'You are text classification Agent. I have a set of open-ended responses, and I need you to classify each one into one of the following category and sub category based on its content. Please fetch keywords as well. The output json format should be {"Category": "","SubCategory":"","Keywords":["",""]}';
converted_data = convert_to_gpt35_format2(sysIns, df)


In [48]:
def convert_to_gpt35_format(sysIns, dataset):
    fine_tuning_data = []
    for _, row in dataset.iterrows():
        row['Category'];
        json_response = '{"Category": "' + row['Category'] + '", "SubCategory": "' + row['SubCategory'] + '", , "Keywords": "' + str(row['Keywords']) + '"}'
        fine_tuning_data.append({
            "messages": [
                {"role": "system", "content": sysIns},
                {"role": "user", "content": row['Answer']},
                {"role": "assistant", "content": json_response}
            ]
        })
    return fine_tuning_data


In [49]:
sysIns = 'You are text classification Agent. I have a set of open-ended responses, and I need you to classify each one into one of the following category and sub category based on its content. Please fetch keywords as well. The output json format should be {"Category": "","SubCategory":"","Keywords":["",""]}';
converted_data = convert_to_gpt35_format(sysIns, df)
converted_data[0]['messages']

[{'role': 'system',
  'content': 'You are text classification Agent. I have a set of open-ended responses, and I need you to classify each one into one of the following category and sub category based on its content. Please fetch keywords as well. The output json format should be {"Category": "","SubCategory":"","Keywords":["",""]}'},
 {'role': 'user',
  'content': "I was looking for a specific product, but it was out of stock for weeks. It�s frustrating that the product range seems broad, but the actual availability is limited. Stock updates weren't clear, and I had to keep checking back to see when the item would be restocked."},
 {'role': 'assistant',
  'content': '{"Category": "Assortment / Products", "SubCategory": "Product availability", , "Keywords": "[\'limited stock\', \'out of stock\', \'availability issues\']"}'}]

In [50]:
import json
converted_data[0]['messages'][-1]['content']
#json.loads(converted_data[0]['messages'][1]['content'])

'{"Category": "Assortment / Products", "SubCategory": "Product availability", , "Keywords": "[\'limited stock\', \'out of stock\', \'availability issues\']"}'

In [51]:
from sklearn.model_selection import train_test_split

# Stratified splitting. Assuming 'Top Category' can be used for stratification
train_data, val_data = train_test_split(
    converted_data,
    test_size=0.2,
    stratify=df['Category'],
    random_state=42  # for reproducibility
)


In [52]:
type(train_data[0])


dict

In [53]:
def write_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for entry in data:
            json.dump(entry, file)
            file.write('\n')


training_file_name = "train.jsonl"
validation_file_name = "val.jsonl"

write_to_jsonl(train_data, training_file_name)
write_to_jsonl(val_data, validation_file_name)

In [54]:
from openai import OpenAI
from google.colab import userdata

client = OpenAI(api_key=userdata.get('cysOpenAIClass'))

In [55]:
training_file = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
validation_file = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)

print("Training file id:", training_file.id)
print("Validation file id:", validation_file.id)

Training file id: file-RHjUeZ3gcX5robUcI7p1ck7N
Validation file id: file-aX1r8IWkjR2WoXEmtJthW6mu


In [56]:
suffix_name = "cys_text_class1"

response = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    validation_file=validation_file.id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
)
response

FineTuningJob(id='ftjob-Vm1EFEyXyiTZbKEDMDfbAbBp', created_at=1730810753, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-RGh6tdzEQ4IIA3FCzFeLzqSL', result_files=[], seed=424548656, status='validating_files', trained_tokens=None, training_file='file-RHjUeZ3gcX5robUcI7p1ck7N', validation_file='file-aX1r8IWkjR2WoXEmtJthW6mu', estimated_finish=None, integrations=[], user_provided_suffix='cys_text_class1')

In [57]:
client.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-Vm1EFEyXyiTZbKEDMDfbAbBp', created_at=1730810753, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-RGh6tdzEQ4IIA3FCzFeLzqSL', result_files=[], seed=424548656, status='validating_files', trained_tokens=None, training_file='file-RHjUeZ3gcX5robUcI7p1ck7N', validation_file='file-aX1r8IWkjR2WoXEmtJthW6mu', estimated_finish=None, integrations=[], user_provided_suffix='cys_text_class1'), FineTuningJob(id='ftjob-Er7d54PCQFFC1cNM2QeKQ8c4', created_at=1730810450, error=Error(code='invalid_training_file', message='The job failed due to an invalid training file. Invalid file format. Line 1, key "messages": The last message must be from the assistant', param='training_file'), fine_tuned_model=None, finished_at=None, hyperpar

In [63]:
response2 = client.fine_tuning.jobs.retrieve("ftjob-Vm1EFEyXyiTZbKEDMDfbAbBp")
response2

FineTuningJob(id='ftjob-Vm1EFEyXyiTZbKEDMDfbAbBp', created_at=1730810753, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:cys-group:cys-text-class1:AQDM8kMH', finished_at=1730811230, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-RGh6tdzEQ4IIA3FCzFeLzqSL', result_files=['file-hUwdIiunpo4Iw1rxTXzwRM21'], seed=424548656, status='succeeded', trained_tokens=23880, training_file='file-RHjUeZ3gcX5robUcI7p1ck7N', validation_file='file-aX1r8IWkjR2WoXEmtJthW6mu', estimated_finish=None, integrations=[], user_provided_suffix='cys_text_class1')

In [65]:
fine_tuned_model_id = response2.fine_tuned_model
print("\nFine-tuned model id:", fine_tuned_model_id)


Fine-tuned model id: ft:gpt-3.5-turbo-0125:cys-group:cys-text-class1:AQDM8kMH


In [69]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def format_test(row):
    formatted_message = [{"role": "user", "content": row['Answer']}]
    return formatted_message


def predict(test_messages, fine_tuned_model_id):
    response = client.chat.completions.create(model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=50)
    return response.choices[0].message.content

In [70]:
def store_predictions(test_df, fine_tuned_model_id):
    print("fine_tuned_model_id",fine_tuned_model_id)
    test_df['Prediction'] = None

    for index, row in test_df.iterrows():
        test_message = format_test(row)
        prediction_result = predict(test_message, fine_tuned_model_id)
        test_df.at[index, 'Prediction'] = prediction_result

    test_df.to_csv("predictions.csv")

In [71]:
test_df = pd.read_json("/content/data_to_test_after_training.json")
store_predictions(test_df, fine_tuned_model_id)

# test model

fine_tuned_model_id ft:gpt-3.5-turbo-0125:cys-group:cys-text-class1:AQDM8kMH
